In [10]:
import numpy as np
from clusterer import *
import pandas as pd
import os
from sklearn.cluster import DBSCAN
from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [11]:
import numpy as np

In [12]:
class mk_clusters(object):

    def processLabels(self, hit):
        #helix transform equations
        hit['rt'] = np.sqrt(hit.x**2+hit.y**2)
        hit['a0'] = np.arctan2(hit.y,hit.x)
        hit['r'] = np.sqrt(hit.x**2+hit.y**2+hit.z**2)
        hit['z1'] = hit.z/hit['rt'] 
        hit['z2'] = hit.z/hit['r']
        dz0 = -0.00070
        stepdz = 0.00001
        #Stepeps is found out by using the average distances amongst data using KNN distances 
        stepeps = 0.000005
        inv = 1
        for i in tqdm(range(100)):
            inv = inv * -1
            dz = inv*(dz0 + i*stepdz)
            hit['a1'] = hit['a0']+dz*hit['z']*np.sign(hit['z'].values)
            hit['sina1'] = np.sin(hit['a1'])
            hit['cosa1'] = np.cos(hit['a1'])
            scaler = StandardScaler()
            df = scaler.fit_transform(hit[['sina1','cosa1','z1','z2']].values)
            #Because Scaling is important foreuclidan data
            sc = np.array([1.0,1.0,0.4,0.4])
            for j in range(np.shape(df)[1]):
                df[:,j] *= sc[j]
            clusters = DBSCAN(eps=0.0035+i*stepeps,min_samples=1,metric='euclidean',n_jobs=8).fit(df).labels_
            #Recursively merging the clusteres and updating their values
            if i==0:
                hit['s1']= clusters
                hit['N1'] = hit.groupby('s1')['s1'].transform('count')
                print(hit['N1'])
                print("###########################")
            else:
                hit['s2'] = clusters
                hit['N2'] = hit.groupby('s2')['s2'].transform('count')
                print(hit['N2'])
                print("###########################")
                max_s1 = hit['s1'].max()
                cond = np.where(((hit['N2'].values>hit['N1'].values) & (hit['N2'].values<20)))
                
                print("#######printing cond#########")
                print(cond)
                print("###########################")
                s1 = hit['s1'].values
                s1[cond] = hit['s2'].values[cond]+max_s1
                hit['s1'] = s1
                hit['s1'] = hit['s1'].astype('int64')
                self.clusters = hit['s1'].values
                hit['N1'] = hit.groupby('s1')['s1'].transform('count')
                
                print("#######hit s1#########")
                
                print(hit['s1'].values)
        return hit['s1'].values
    
    def predict(self, hits):         
        self.clusters = self.processLabels(hits)      
        return self.clusters

In [3]:
path_to_train = "train_100_events"
event_name = "event000001000"
hits, cells, particles, truth = load_event(os.path.join(path_to_train, event_name))

In [5]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission  

In [13]:
train_submissions = []
scores = []
for event_id, hits, cells, particles, truth in load_dataset(path_to_train, skip=0, nevents=5):
    model = Clusterer()
    print(event_id)
    print(hits)
    labels = model.predict(hits)
    print(labels)

    one_submission = create_one_event_submission(event_id, hits, labels)
    print(one_submission)
    train_submissions.append(one_submission)

    score = score_event(truth, one_submission)
    scores.append(score)

    print("Score for event %d: %.8f" % (event_id, score))
print('Mean score: %.8f' % (np.mean(scores)))

1000
        hit_id           x           y       z  volume_id  layer_id  module_id
0            1  -64.409897   -7.163700 -1502.5          7         2          1
1            2  -55.336102    0.635342 -1502.5          7         2          1
2            3  -83.830498   -1.143010 -1502.5          7         2          1
3            4  -96.109100   -8.241030 -1502.5          7         2          1
4            5  -62.673599   -9.371200 -1502.5          7         2          1
5            6  -57.068699   -8.177770 -1502.5          7         2          1
6            7  -73.872299   -2.578900 -1502.5          7         2          1
7            8  -63.853500  -10.868400 -1502.5          7         2          1
8            9  -97.254799  -10.889100 -1502.5          7         2          1
9           10  -90.292900   -3.269370 -1502.5          7         2          1
10          11  -59.182999   -0.670508 -1502.5          7         2          1
11          12  -42.623901  -10.661200 -1502.5 


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

0         1
1         1
2         1
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19        1
20        1
21        1
22        1
23        1
24        1
25        1
26        1
27        1
28        1
29        1
         ..
120909    2
120910    3
120911    1
120912    1
120913    1
120914    1
120915    1
120916    3
120917    2
120918    2
120919    2
120920    1
120921    1
120922    1
120923    1
120924    2
120925    2
120926    2
120927    2
120928    2
120929    3
120930    1
120931    4
120932    1
120933    1
120934    1
120935    1
120936    1
120937    3
120938    7
Name: N1, Length: 120939, dtype: int64
###########################



  1%|▊                                                                                 | 1/100 [00:01<02:02,  1.24s/it]

0         1
1         1
2         1
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19        1
20        1
21        1
22        2
23        1
24        1
25        1
26        1
27        1
28        2
29        1
         ..
120909    4
120910    2
120911    2
120912    1
120913    2
120914    4
120915    1
120916    2
120917    1
120918    2
120919    1
120920    1
120921    2
120922    1
120923    1
120924    1
120925    1
120926    2
120927    1
120928    1
120929    1
120930    2
120931    1
120932    1
120933    1
120934    2
120935    2
120936    1
120937    1
120938    3
Name: N2, Length: 120939, dtype: int64
###########################


...

0         1
1         1
2         1
3         1
4         2
5         1
6         1
7         1
8         1
9         1
10        2
11        1
12        2
13        1
14        1
15        1
16        1
17        1
18        1
19        1
20        2
21        1
22        1
23        4
24        2
25        1
26        1
27        2
28        2
29        1
         ..
125474    1
125475    3
125476    1
125477    3
125478    8
125479    1
125480    2
125481    1
125482    4
125483    1
125484    3
125485    1
125486    1
125487    3
125488    1
125489    8
125490    8
125491    1
125492    8
125493    4
125494    2
125495    1
125496    2
125497    1
125498    2
125499    2
125500    3
125501    1
125502    1
125503    1
Name: N2, Length: 125504, dtype: int64
###########################



 19%|███████████████▍                                                                 | 19/100 [00:17<01:19,  1.02it/s]

0         1
1         2
2         2
3         1
4         2
5         2
6         1
7         1
8         1
9         2
10        1
11        1
12        2
13        1
14        1
15        2
16        1
17        1
18        1
19        2
20        1
21        1
22        2
23        1
24        2
25        1
26        1
27        1
28        1
29        1
         ..
125474    2
125475    1
125476    1
125477    1
125478    3
125479    1
125480    1
125481    1
125482    1
125483    1
125484    4
125485    1
125486    1
125487    2
125488    1
125489    3
125490    3
125491    2
125492    4
125493    1
125494    3
125495    3
125496    1
125497    1
125498    1
125499    6
125500    2
125501    1
125502    3
125503    2
Name: N2, Length: 125504, dtype: int64
###########################



 20%|████████████████▏                                                                | 20/100 [00:18<01:20,  1.00s/it]

KeyboardInterrupt: 

In [ ]:
path_to_test = "test"
test_submission = []

for event_id, hits, cells in load_dataset(path_to_test, parts=['hits', 'cells']):

    model = mk_clusters()
    labels = model.predict(hits)

    one_submission = create_one_event_submission(event_id, hits, labels)
    test_submission.append(one_submission)
    print('Event ID: ', event_id)

submission = pd.concat(test_submission, axis=0)
submission.to_csv('submission1.csv', index=False)